In [1]:
from __future__ import print_function # Python 2/3 compatibility
import cv2 # Import the OpenCV library
import numpy as np # Import Numpy library
from scipy.spatial.transform import Rotation as R
import math # Math library
from numpy.linalg import inv

/home/peerajak/venv/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/peerajak/venv/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/peerajak/venv/lib/python3.8/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/peerajak/venv/lib/python3.8/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
# Dictionary that was used to generate the ArUco marker
aruco_dictionary_name = "DICT_6X6_250"
 
# The different ArUco dictionaries built into the OpenCV library. 
ARUCO_DICT = {
  "DICT_4X4_50": cv2.aruco.DICT_4X4_50,
  "DICT_4X4_100": cv2.aruco.DICT_4X4_100,
  "DICT_4X4_250": cv2.aruco.DICT_4X4_250,
  "DICT_4X4_1000": cv2.aruco.DICT_4X4_1000,
  "DICT_5X5_50": cv2.aruco.DICT_5X5_50,
  "DICT_5X5_100": cv2.aruco.DICT_5X5_100,
  "DICT_5X5_250": cv2.aruco.DICT_5X5_250,
  "DICT_5X5_1000": cv2.aruco.DICT_5X5_1000,
  "DICT_6X6_50": cv2.aruco.DICT_6X6_50,
  "DICT_6X6_100": cv2.aruco.DICT_6X6_100,
  "DICT_6X6_250": cv2.aruco.DICT_6X6_250,
  "DICT_6X6_1000": cv2.aruco.DICT_6X6_1000,
  "DICT_7X7_50": cv2.aruco.DICT_7X7_50,
  "DICT_7X7_100": cv2.aruco.DICT_7X7_100,
  "DICT_7X7_250": cv2.aruco.DICT_7X7_250,
  "DICT_7X7_1000": cv2.aruco.DICT_7X7_1000,
  "DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL
}

In [3]:
# Side length of the ArUco marker in meters 
aruco_marker_side_length = 0.026

In [4]:
def euler_from_quaternion(x, y, z, w):
  """
  Convert a quaternion into euler angles (roll, pitch, yaw)
  roll is rotation around x in radians (counterclockwise)
  pitch is rotation around y in radians (counterclockwise)
  yaw is rotation around z in radians (counterclockwise)
  """
  t0 = +2.0 * (w * x + y * z)
  t1 = +1.0 - 2.0 * (x * x + y * y)
  roll_x = math.atan2(t0, t1)
      
  t2 = +2.0 * (w * y - z * x)
  t2 = +1.0 if t2 > +1.0 else t2
  t2 = -1.0 if t2 < -1.0 else t2
  pitch_y = math.asin(t2)
      
  t3 = +2.0 * (w * z + x * y)
  t4 = +1.0 - 2.0 * (y * y + z * z)
  yaw_z = math.atan2(t3, t4)
      
  return roll_x, pitch_y, yaw_z # in radians

In [5]:
def main():
  global p98
  global p98_r
  global p40
  global p40_r
  """
  Main method of the program.
  """
  # Check that we have a valid ArUco marker
  if ARUCO_DICT.get(aruco_dictionary_name, None) is None:
    print("[INFO] ArUCo tag of '{}' is not supported".format(
      args["type"]))
    sys.exit(0)
 
  # Load the camera parameters from the saved file
  #cv_file = cv2.FileStorage( camera_calibration_parameters_filename, cv2.FILE_STORAGE_READ) 
  #mtx = cv_file.getNode('K').mat()
  #dst = cv_file.getNode('D').mat()
  #cv_file.release()

  mtx_np = np.array([ [759.895784, 0.000000, 312.753105],[0.000000, 762.113647, 214.923553], [0., 0., 1.]], np.float32)
  mtx = mtx_np
  dst_np = np.array([0.062948, -0.273568, 0.005933, -0.001056, 0.000000], np.float32)   
  dst = dst_np
  # Load the ArUco dictionary
  print("[INFO] detecting '{}' markers...".format(
    aruco_dictionary_name))
  this_aruco_dictionary = cv2.aruco.Dictionary_get(ARUCO_DICT[aruco_dictionary_name])
  this_aruco_parameters = cv2.aruco.DetectorParameters_create()
   
  # Start the video stream

   
 
  # Capture frame-by-frame
  # This method returns True/False as well
  # as the video frame.
  streamImage = cv2.imread('data/original_image.jpg')
  detectingImage = streamImage.copy() 

  # Detect ArUco markers in the video frame
  (corners, marker_ids, rejected) = cv2.aruco.detectMarkers(
      detectingImage , this_aruco_dictionary, parameters=this_aruco_parameters,
      cameraMatrix=mtx, distCoeff=dst)

  print('corners ',corners)
  print('marker_ids', marker_ids)

  # Check that at least one ArUco marker was detected
  if marker_ids is not None: 
      # Draw a square around detected markers in the video frame
      cv2.aruco.drawDetectedMarkers(detectingImage , corners, marker_ids)

      # Get the rotation and translation vectors
      rvecs, tvecs, obj_points = cv2.aruco.estimatePoseSingleMarkers(
        corners,
        aruco_marker_side_length,
        mtx,
        dst)
      # Print the pose for the ArUco marker
      # The pose of the marker is with respect to the camera lens frame.
      # Imagine you are looking through the camera viewfinder, 
      # the camera lens frame's:
      # x-axis points to the right
      # y-axis points straight down towards your toes
      # z-axis points straight ahead away from your eye, out of the camera
      for i, marker_id in enumerate(marker_ids):

        # Store the translation (i.e. position) information
        transform_translation_x = tvecs[i][0][0]
        transform_translation_y = tvecs[i][0][1]
        transform_translation_z = tvecs[i][0][2]

        # Store the rotation information
        rotation_matrix = np.eye(4)
        rotation_matrix[0:3, 0:3] = cv2.Rodrigues(np.array(rvecs[i][0]))[0]
        r = R.from_matrix(rotation_matrix[0:3, 0:3])
        quat = r.as_quat()   

        # Quaternion format     
        transform_rotation_x = quat[0] 
        transform_rotation_y = quat[1] 
        transform_rotation_z = quat[2] 
        transform_rotation_w = quat[3] 

        # Euler angle format in radians
        roll_x, pitch_y, yaw_z = euler_from_quaternion(transform_rotation_x, 
                                                       transform_rotation_y, 
                                                       transform_rotation_z, 
                                                       transform_rotation_w)

        # roll_x = math.degrees(roll_x)
        # pitch_y = math.degrees(pitch_y)
        # yaw_z = math.degrees(yaw_z)
        # print("transform_translation_x: {}".format(transform_translation_x))
        # print("transform_translation_y: {}".format(transform_translation_y))
        # print("transform_translation_z: {}".format(transform_translation_z))
        # print("roll_x: {}".format(roll_x))
        # print("pitch_y: {}".format(pitch_y))
        # print("yaw_z: {}".format(yaw_z))
        # print()
        
        print("id {} ".format(marker_id))
        print("rotation matrix",rotation_matrix[0:3, 0:3] )
        print("translation matrix",[transform_translation_x ,transform_translation_y,transform_translation_z])
        if marker_id == 98:
            p98 = tvecs[i][0][0:3]
            p98_r = rotation_matrix[0:3, 0:3]
            print('set 98')
        if marker_id == 40:
            p40 = tvecs[i][0][0:3]
            p40_r = rotation_matrix[0:3, 0:3]  
            print('set 40')
        print("============")

        # Draw the axes on the marker
        cv2.aruco.drawAxis(detectingImage , mtx, dst, rvecs[i], tvecs[i], 0.05)

  # Display the resulting frame
  cv2.imshow('frame',detectingImage )
  while cv2.waitKey(5000) & 0xFF == ord('q'):
      break


  
  # Close down the video stream
  #cap.release()
  #cv2.destroyAllWindows()


In [6]:
   
if __name__ == '__main__':
  print(__doc__)
  main()

Automatically created module for IPython interactive environment
[INFO] detecting 'DICT_6X6_250' markers...
corners  [array([[[293., 291.],
        [292., 336.],
        [248., 336.],
        [249., 291.]]], dtype=float32), array([[[447., 309.],
        [402., 309.],
        [401., 264.],
        [446., 264.]]], dtype=float32), array([[[223., 215.],
        [222., 258.],
        [178., 259.],
        [181., 215.]]], dtype=float32), array([[[397., 150.],
        [398., 192.],
        [354., 193.],
        [354., 150.]]], dtype=float32), array([[[266.,  91.],
        [266.,  50.],
        [308.,  50.],
        [307.,  91.]]], dtype=float32), array([[[446.,  26.],
        [448.,  67.],
        [406.,  68.],
        [404.,  27.]]], dtype=float32)]
marker_ids [[ 98]
 [124]
 [ 40]
 [ 23]
 [ 62]
 [203]]
id [98] 
rotation matrix [[-0.01803717  0.98100838  0.1931249 ]
 [ 0.99875872  0.02664799 -0.04208221]
 [-0.04642939  0.19212614 -0.98027132]]
translation matrix [-0.024660402520593854, 0.0574

In [7]:
# This might be wrong or right. I was expecting z component of p40__p98 to be nearly zero
# but this is because the camera calibration was done on the laptop computer

T_c_p98 = p98_r
T_c_p40 = p40_r

p40__p98 = np.dot(T_c_p98, p40 - p98  )
print(p40__p98)

p98__p40 = np.dot(T_c_p40, p98 - p40  )
print(p98__p40)

[-0.0424999  -0.04266978 -0.01011032]
[ 0.04437287  0.04058295 -0.01064155]
